In [ ]:
# | default_exp opt.objectives

In [ ]:
# | export
import operator
import torch
from abc import ABC
import fastcore.all as fc
from torch import nn
from translucid.opt.hooks import *
from translucid.utils import *

In [ ]:
# | export

class Objective(ABC):
    def __init__(self, loss_fn, repr=None):
        self.loss_fn, self.repr = loss_fn, repr

    def __repr__(self):
        return self.repr or self.__class__.__name__

    @staticmethod
    def _store_hook(hook, module, input, output):
        hook.value = output

    def __call__(self, model: nn.Module, x):
        with NamedHooks(model.named_modules(), fn=self._store_hook) as hooks: model(x)
        return self.eval(hooks)

    def eval(self, hooks):
        return self.loss_fn(hooks)

    def _op(self, other, op):
        if isinstance(other, (int, float)):
            fn = lambda hooks: op(self.eval(hooks), other)
        elif isinstance(other, Objective):
            fn = lambda hooks: op(self.eval(hooks), other.eval(hooks))
        return Objective(fn, repr=f'{self} {op2str(op)} {other}')

    def __add__(self, other): return self._op(other, operator.add)
    def __radd__(self, other): return self + other
    def __mul__(self, other): return self._op(other, operator.mul)
    def __rmul__(self, other): return self * other
    def __sub__(self, other): return self._op(other, operator.sub)
    def __rsub__(self, other): return -self + other
    def __pow__(self, other): return self._op(other, operator.pow)
    def __rpow__(self, other): return Objective(lambda hooks: other ** self.eval(hooks))
    def __truediv__(self, other): return self._op(other, operator.truediv)
    def __rtruediv__(self, other): return self ** -1 * other
    def __neg__(self): return self * -1


class ModuleObjective(Objective):
    def __init__(self, module_name, loss_fn, repr):
        super().__init__(loss_fn, repr)
        self.module_name = module_name

    def eval(self, hooks):
        return self.loss_fn(hooks[self.module_name].value)

In [ ]:
# | export

def neuron(layer, n_channel, x=None, y=None, batch=None):
    """
    Visualize a specific neuron within a given channel. 
    If no (x, y) coordinates are provided, the neuron at the center is chosen. 
    For layers with even width and height, the bottom-right neuron of the central 2x2 region is selected by default.
    """
    def loss_fn(out):
        assert len(out.shape) == 4
        x_pos, y_pos = x or out.shape[-1] // 2, y or out.shape[-2] // 2
        return out[fc.ifnone(batch, slice(batch)), n_channel, y_pos, x_pos].mean()
    r = to_repr(f"n{n_channel}_neuron", layer=layer, x=x, y=y, batch=batch)
    return ModuleObjective(layer, loss_fn, repr=r)


def channel(layer, n_channel, batch=None):
    """Visualize an entire channel of a layer."""
    def loss_fn(out):
        return out[fc.ifnone(batch, slice(batch)), n_channel].mean()
    r = to_repr(f"n{n_channel}_channel", layer=layer, batch=batch)
    return ModuleObjective(layer, loss_fn, repr=r)

In [ ]:
x = torch.randn(3, 1, 28, 28)
obj = neuron('1', 5, batch=0) + channel('1', 1, batch=1) + channel('2', 2, batch=2)
obj

n5_neuron(layer='1', batch=0) + n1_channel(layer='1', batch=1) + n2_channel(layer='2', batch=2)

In [ ]:
model = nn.Sequential(
    nn.Conv2d(1, 32, 3),
    nn.ReLU(),
    nn.Conv2d(32, 64, 3))

obj(model, x)

tensor(0.0591, grad_fn=<AddBackward0>)